# Fine tune LLM on database

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [1]:
APPLY = True
TABLE_NAME = 'sample_llm_finetuning'

In [2]:
from superduper import superduper

db = superduper('mongomock://test_db')

2025-Mar-26 20:51:07.84| INFO     | Duncans-MBP.fritz.box| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Mar-26 20:51:07.89| INFO     | Duncans-MBP.fritz.box| superduper.base.artifacts:178  | Creating artifact store directory
2025-Mar-26 20:51:07.91| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:51   | Building Data Layer
2025-Mar-26 20:51:07.91| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:68   | Data Layer built
2025-Mar-26 20:51:07.92| INFO     | Duncans-MBP.fritz.box| superduper.backends.base.cluster:112  | Cluster initialized in 0.00 seconds.
2025-Mar-26 20:51:07.92| INFO     | Duncans-MBP.fritz.box| superduper.base.build:150  | Configuration: 
 +----------------+-------------------------------+
| Configuration  |             Value             |
+----------------+-------------------------------+
|  Data Backend  |      mongomock://test_db      |
| Artifact Store | filesystem://./artifact_store |
|     Cache      |           in-process  

<!-- TABS -->
## Get LLM Finetuning Data

The following are examples of training data in different formats.

In [3]:
from datasets import load_dataset
from superduper.base.document import Document

def getter():

    dataset_name = "timdettmers/openassistant-guanaco"
    dataset = load_dataset(dataset_name)
    
    train_dataset = dataset["train"]
    eval_dataset = dataset["test"]
    
    train_documents = [{**example, "_fold": "train"} for example in train_dataset][:10]
    eval_documents = [{**example, "_fold": "valid"} for example in eval_dataset][:5]
    
    data = train_documents + eval_documents

    return data

[2025-03-26 20:51:08] datasets INFO PyTorch version 2.5.1 available.


In [4]:
if APPLY:
    data = getter()

Repo card metadata block was not found. Setting CardData to empty.
[2025-03-26 20:51:09] huggingface_hub.repocard WARNING Repo card metadata block was not found. Setting CardData to empty.


We can define different training parameters to handle this type of data.

In [5]:
transform = None
key = ('text')
training_kwargs=dict(dataset_text_field="text")

Example input_text and output_text

In [6]:
if APPLY:
    d = data[0]
    input_text, output_text = d["text"].rsplit("### Assistant: ", maxsplit=1)
    input_text += "### Assistant: "
    output_text = output_text.rsplit("### Human:")[0]
    print("Input: --------------")
    print(input_text)
    print("Response: --------------")
    print(output_text)

Input: --------------
### Human: Can you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.### Assistant: 
Response: --------------
"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.

Recent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [7]:
from superduper import Table

table_or_collection = db[TABLE_NAME]

if APPLY:
    db.apply(
        Table(
            TABLE_NAME,
            fields={'text': 'str', '_fold': 'str'},
        ),
        force=True
    )
    
    table_or_collection.insert(data)

select = table_or_collection

2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:408  | Load (('Table', 'sample_llm_finetuning')) from metadata...
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | Found these changes and/ or additions that need to be made:
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:91   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | METADATA EVENTS:
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:93   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:104  | [0]: Table:sample_llm_finetuning:4d10ae1f24e5b7094036ec76a1fc738b: create
2025-Mar-26 20:51:11.76| INFO     | Duncans-MBP.fritz.bo

## Select a Model

In [8]:
model_name = "Qwen/Qwen2.5-0.5B"
model_kwargs = dict()
tokenizer_kwargs = dict()

<!-- TABS -->
## Build A Trainable LLM

**Create an LLM Trainer for training**

The parameters of this LLM Trainer are basically the same as `transformers.TrainingArguments`, but some additional parameters have been added for easier training setup.

In [9]:
from superduper_transformers import LLM, LLMTrainer

trainer = LLMTrainer(
    identifier="llm-finetune-trainer",
    output_dir="output/finetune",
    overwrite_output_dir=True,
    num_train_epochs=2,
    save_total_limit=10,
    logging_steps=1,
    evaluation_strategy="steps",
    save_steps=100,
    eval_steps=100,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    key=key,
    select=select,
    transform=transform,
    training_kwargs=training_kwargs,
    use_lora=True,
)

Create a trainable LLM model and add it to the database, then the training task will run automatically.

In [10]:
llm = LLM(
    identifier="llm",
    model_name_or_path=model_name,
    trainer=trainer,
    model_kwargs=model_kwargs,
    tokenizer_kwargs=tokenizer_kwargs,
)

In [11]:
if APPLY:
    db.apply(llm, force=True)

2025-Mar-26 20:51:13.56| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:408  | Load (('LLM', 'llm')) from metadata...
2025-Mar-26 20:51:13.56| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:408  | Load (('LLMTrainer', 'llm-finetune-trainer')) from metadata...
2025-Mar-26 20:51:13.57| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | Found these changes and/ or additions that need to be made:
2025-Mar-26 20:51:13.57| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:91   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:13.57| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | METADATA EVENTS:
2025-Mar-26 20:51:13.57| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:93   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:13.57| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:10

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use mps:0


2025-Mar-26 20:51:15.34| INFO     | Duncans-MBP.fritz.box| superduper_transformers.training:290  | Start training, experiment_id: 36f73c9bf63b6ec25ee7488c8a83bc8db9cfc91b
2025-Mar-26 20:51:15.35| INFO     | Duncans-MBP.fritz.box| superduper_transformers.training:493  | Start training LLM model
2025-Mar-26 20:51:15.35| INFO     | Duncans-MBP.fritz.box| superduper_transformers.training:494  | training_args: LLMTrainer(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
bits=None,
compute_kwargs={},
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory

/Users/dodo/.pyenv/versions/3.10.13/envs/superduper-3.10/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


2025-Mar-26 20:51:16.08| INFO     | Duncans-MBP.fritz.box| superduper_transformers.training:525  | tokenizer_kwargs: %s {'pretrained_model_name_or_path': 'Qwen/Qwen2.5-0.5B'}
2025-Mar-26 20:51:16.38| INFO     | Duncans-MBP.fritz.box| superduper_transformers.training:540  | Preparing LoRA training
trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826


...

Step,Training Loss,Validation Loss


2025-Mar-26 20:51:26.82| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:408  | Load (('LLM', 'llm')) from metadata...
2025-Mar-26 20:51:26.84| INFO     | Duncans-MBP.fritz.box| superduper.base.datalayer:408  | Load (('LLMTrainer', 'llm-finetune-trainer')) from metadata...
2025-Mar-26 20:51:26.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.artifacts:243  | File ./artifact_store/4f53cda18c2baa0c0354bb5f9a3ecbe5ed12ab4d8e11ba873c2f11161202b945 already exists
2025-Mar-26 20:51:26.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.artifacts:243  | File ./artifact_store/e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855 already exists
2025-Mar-26 20:51:26.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.artifacts:243  | File ./artifact_store/b7595e2a863957fc6c225ed540d335dd1e859dd3032451c2e11bceb0329defa6 already exists
2025-Mar-26 20:51:26.87| WARNING  | Duncans-MBP.fritz.box| superduper.base.artifacts:243  | File ./artifact_store/fe2d01de4181686ab3

llm
├── llm
│   └── status: update
└── llm-finetune-trainer
    └── status: update

2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:89   | Found these changes and/ or additions that need to be made:
2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:91   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:92   | METADATA EVENTS:
2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:93   | ----------------------------------------------------------------------------------------------------
2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:100  | [0]: LLMTrainerllm-finetune-trainer:2bd3f6003ba38731e4c8d18972dd920a: update ~ [2]
2025-Mar-26 20:51:26.88| INFO     | Duncans-MBP.fritz.box| superduper.base.apply:100  | [1]: Checkpoint:36f73c9bf63b6ec25ee7488c8a83bc8db9cfc91b:e2973c41a7d01b0262f5dcadea1649dd: create ~ [2]
2025-M

Device set to use mps:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausal

In [12]:
from superduper import Template, Table, Application
from superduper.components.dataset import RemoteData

llm.trainer.use_lora = "<var:use_lora>"
llm.trainer.num_train_epochs = "<var:num_train_epochs>"

app = Application(identifier="llm", components=[llm])

t = Template(
    'llm-finetune',
    template=app,
    substitutions={
        TABLE_NAME: 'table_name',
        model_name: 'model_name',
    },
    default_tables=[
        Table(
            'sample_llm_finetuning',
            fields={'x': 'str', 'y': 'int'},
            data=RemoteData(
                'llm_finetuning',
                getter=getter,
            ),
        )
    ],
    template_variables=['table_name', 'model_name', 'use_lora', 'num_train_epochs'],
    types={
        'collection': {
            'type': 'str',
            'default': 'dataset',
        },
        'model_name': {
            'type': 'str',
            'default': 'Qwen/Qwen2.5-0.5B',
        },
        'use_lora': {
            'type': 'bool',
            'default': True,
        },
        'num_train_epochs': {
            'type': 'int',
            'default': 3
        },
        'table_name': {
            'type': 'str',
            'default': 'sample_llm_finetuning',
        },
    },
    db=db,
)

t.export('.')

## Load the trained model
There are two methods to load a trained model:

- **Load the model directly**: This will load the model with the best metrics (if the transformers' best model save strategy is set) or the last version of the model.
- **Use a specified checkpoint**: This method downloads the specified checkpoint, then initializes the base model, and finally merges the checkpoint with the base model. This approach supports custom operations such as resetting flash_attentions, model quantization, etc., during initialization.

In [ ]:
if APPLY:
    llm = db.load("LLM", "llm")

In [ ]:
if APPLY:
    print(llm.predict(input_text, max_new_tokens=200))